In [ ]:
# Import Selenium and its sub libraries
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import re

In [ ]:
url="https://openlibrary.org/"

In [ ]:
#Create lists
books_list = []
author_list=[]
link_list=[]
pdf_list=[]

In [ ]:
def extract_page_data(driver):
    #Grab all books
    books=driver.find_elements("class name", "searchResultItem")
    #print(len(books))

    #Create loop to itterate on the list
    for b in range(len(books)):
        try:          
            #Extracting title
            title = books[b].find_element("class name","booktitle").text
            #print(title)
        
            #Extraction of authors
            authors=books[b].find_element("class name","bookauthor").text
            #print(authors)
        
            #Extraction of link
            link = books[b].find_element("link text",title).get_attribute('href')
            #print(link)
        
                  
            #Add item to the respective lists
            books_list.append(title)
            author_list.append(authors)
            link_list.append(link)
            sleep(2)  
        
        except:
            pass
                       
            
        # Not all items have pdf
        try:
            #Extraction of pdf
            pdf= books[b].find_element("xpath",".//a[(@title = 'Read ebook from Internet Archive')]").get_attribute('href')
            #print(pdf) 
            pdf_list.append(pdf)
        except:
            pdf_list.append("NONE")
            continue

In [ ]:
driver = webdriver.Chrome()
driver.get(url)

#Create lists to store the data
books_list = []
author_list=[]
link_list=[]
pdf_list=[]

#Step 1: identify search box
search_query = driver.find_element("name","q")

#Step 2: search a topic
search_query.send_keys('Venice')
search_query.send_keys(Keys.RETURN)
#Delay
sleep(1)

#Step 3: Execute the function for each page
for page_num in range(0,3):
    
    #Execute the function 
    extract_page_data(driver)
        
    #Locate the "Next" button on the webpage and click it to navigate to the next page
    driver.find_element("xpath", "//*[@id='searchResults']/div[2]/a[10]").click()

#Close the webpage
driver.quit()

In [ ]:
#Import the library
import csv 

#combine the lists
data=zip(books_list,author_list,link_list)

#Write all the data to a csv
with open('openlibrarycatalogue.csv', 'w',newline='') as csvfile: 
    writer = csv.writer(csvfile) 
    for row in data:
        writer.writerow(row)
        
#Write only links to a csv
with open('onlylinks.csv', 'w',newline='') as csvfile: 
    writer = csv.writer(csvfile) 
    for link in link_list:
        #print(row)
        writer.writerow([link])

In [ ]:
#import a new library
import requests


#read the produced csv and iterate through the list
with open("onlylinks1.csv", 'r',encoding='utf-8') as file:
    csvreader = csv.reader(file)    
    for url in csvreader:       
                
        #open each link
        driver = webdriver.Chrome()
        driver.get(url[0])
        sleep(2)
        
        #click on the pdf button
        find_pdf=driver.find_element("xpath","//li//a[@title='Download a PDF from Internet Archive']").get_attribute('href')
        print(find_pdf)

        
        ##########
        
        #define a counter for naming 
        i=0
        
        #Check if pdf exists and then download



        if ('.pdf' in find_pdf):
            i += 1
            #print("Downloading file: ", i)
 
            # Get response object for link
            response = requests.get(find_pdf)
 
            # Write content in pdf file
            pdf = open("pdf"+str(i)+".pdf", 'wb')
            pdf.write(response.content)
            pdf.close()
       
        #Close the webpage
        driver.quit()